# wps_update_metadata

#### wps_update_metadata is a process that runs the [update_metadata](https://github.com/pacificclimate/climate-explorer-data-prep#update_metadata-update-metadata-in-a-netcdf-file) module of PCIC Climate Explorer Data Preparation Tools. Here, the client will try to connect to a remote Thunderbird instance using the url parameter.

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

from birdy import WPSClient
from pkg_resources import resource_filename
import requests
import os
from datetime import datetime
from wps_tools.utils import copy_http_content
from netCDF4 import Dataset
from tempfile import NamedTemporaryFile

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "thunderbird":
    os.chdir('../')

url = 'https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thunderbird/wps'
thunderbird = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [2]:
# NBVAL_IGNORE_OUTPUT
thunderbird.update_metadata?

Signature: thunderbird.update_metadata(netcdf, updates=None, loglevel='INFO')
Docstring:
Update file containing missing, invalid, or incorrectly named global or variable metadata attributes

Parameters
----------
netcdf : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    NetCDF file
updates : string
    The filepath of an updates file that specifies what to do to the metadata it finds in the NetCDF file
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
output : ComplexData:mimetype:`application/x-netcdf`
    Output Netcdf File
File:      ~/Desktop/thunderbird/</home/sangwonl/Desktop/thunderbird/venv/lib/python3.7/site-packages/birdy/client/base.py-2>
Type:      method


#### We can use the docstring to ensure we provide the appropriate parameters.

In [3]:
# run update_metadata
updates = '''
global:
    - institute_id:
    - institute_ID: PCIC
    - address: University House 1 
    - Institution: <- institution
'''
netcdf = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/gdd_annual_CanESM2_rcp85_r1i1p1_1951-2100.nc"
output = thunderbird.update_metadata(updates = updates, netcdf = netcdf)

#### Once the process has completed we can extract the results and ensure it is what we expected.

In [4]:
input_data = Dataset("https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/gdd_annual_CanESM2_rcp85_r1i1p1_1951-2100.nc")

# Expected output metadata determined from the input yaml file
# global:
#     - institute_id:
#     - institute_ID: PCIC
#     - address: University House 1 
#     - Institution: <- institution
expected = {
    "institute_ID": "PCIC",
    "address": "University House 1",
    "Institution": input_data.institution,
}

In [5]:
with NamedTemporaryFile(suffix=".nc", prefix="tmp_copy", dir="/tmp", delete=True) as tmp_file:
    output_data = Dataset(copy_http_content(output.get()[0], tmp_file))

# updated metadata
metadata = {
    "institute_ID": output_data.institute_ID,
    "address": output_data.address,
    "Institution": output_data.Institution,
}

assert metadata == expected